In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
    .appName("gold_star_schema") \
    .config("spark.databricks.delta.autoCompact.enabled", "true") \
    .getOrCreate()

silver= 'silver_Medicare_CMS_PartD'

StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 3, Finished, Available)

In [2]:
#Dimension_YEAR

from pyspark.sql.functions import col
from pyspark.sql.types import DateType

dim_year_df = spark_session.sql(f"SELECT DISTINCT Year, CONCAT(CAST(Year AS String), '-01-01') AS Year_Date_Key FROM {silver}")

dim_year_df = dim_year_df.withColumn("Year_Date_Key", col('Year_Date_Key').cast(DateType()))

dim_year_df.write.mode("overwrite").format("delta").save("Tables/gold_dim_year")

StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 4, Finished, Available)

In [3]:
#Dimension_Geography

dim_geo_df = spark_session.sql(f'''SELECT Prscrbr_City, Prscrbr_City_State, Prscrbr_State_Abrvtn, Prscrbr_State_FIPS, MAX(Year) AS Max_Year, MIN(Year) AS Min_Year,
    row_number() OVER (ORDER BY Prscrbr_State_Abrvtn,Prscrbr_City_State ASC) AS geo_key
    FROM {silver}
    GROUP BY Prscrbr_City,Prscrbr_City_State,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS ''')

dim_geo_df.write.mode("overwrite").format("delta").save("Tables/gold_dim_geography")

StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 5, Finished, Available)

In [4]:
#Dimension_Provider

dim_provider_df = spark_session.sql(f'''SELECT Prscrbr_First_Name
,Prscrbr_Full_Name
,Prscrbr_Last_Org_Name
,Prscrbr_NPI
,Prscrbr_Type
,Prscrbr_Type_Src
,MAX(Year) AS Max_Year
,MIN(Year) AS Min_Year
,row_number() OVER (ORDER BY Prscrbr_Full_Name,Prscrbr_NPI,Prscrbr_Type,Prscrbr_Type_Src ASC) AS provider_key
FROM {silver}
GROUP BY Prscrbr_First_Name,Prscrbr_Full_Name,Prscrbr_Last_Org_Name,Prscrbr_NPI,Prscrbr_Type,Prscrbr_Type_Src''')

dim_provider_df.write.mode("overwrite").format("delta").save("Tables/gold_dim_provider")

StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 6, Finished, Available)

In [5]:
#Dimension_drug

dim_drug_df = spark_session.sql(f'''SELECT Brnd_Name
,Gnrc_Name
,MAX(Year) AS Max_Year
,MIN(Year) AS Min_Year
,row_number() OVER (ORDER BY Brnd_Name,Gnrc_Name ASC) AS drug_key
FROM {silver}
GROUP BY Brnd_Name, Gnrc_Name''')

dim_drug_df.write.mode("overwrite").format("delta").save("Tables/gold_dim_drug")

StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 7, Finished, Available)

In [13]:
drug_costs_star_df = spark_session.sql(f'''SELECT GE65_Bene_Sprsn_Flag
,GE65_Sprsn_Flag
,GE65_Tot_30day_Fills
,GE65_Tot_Benes
,GE65_Tot_Clms
,GE65_Tot_Day_Suply
,GE65_Tot_Drug_Cst
,Tot_30day_Fills
,Tot_Benes
,Tot_Clms
,Tot_Day_Suply
,Tot_Drug_Cst
,Year
,b.drug_key
,c.geo_key
,d.provider_key
FROM {silver} a
LEFT OUTER JOIN gold_dim_drug b ON a.Brnd_Name = b.Brnd_Name AND a.Gnrc_Name = b.Gnrc_Name
LEFT OUTER JOIN gold_dim_geography c ON a.Prscrbr_City_State IS NOT DISTINCT FROM c.Prscrbr_City_State 
LEFT OUTER JOIN gold_dim_provider d ON a.Prscrbr_Full_Name IS NOT DISTINCT FROM d.Prscrbr_Full_Name AND a.Prscrbr_NPI = d.Prscrbr_NPI AND a.Prscrbr_Type IS NOT DISTINCT FROM d.Prscrbr_Type AND a.Prscrbr_Type_Src = d.Prscrbr_Type_Src
;''')

drug_costs_star_df.write.mode("overwrite").format("delta").save("Tables/gold_drug_costs_star")



StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 15, Finished, Available)

AnalysisException: ANALYZE TABLE is not supported for v2 tables.

In [15]:
spark_session.sql('''OPTIMIZE gold_drug_costs_star ZORDER BY (drug_key, provider_key);''')

StatementMeta(, 727cd008-281f-4ca8-a654-69883eaf3dea, 17, Finished, Available)

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de